In [ ]:
from flask import Flask, request, jsonify
from flask_cors import CORS
from sentence_transformers import SentenceTransformer
from qdrant_client import QdrantClient
import logging

# Configure logging
logging.basicConfig(level=logging.INFO)

# Initialize Flask app
app = Flask(__name__)
CORS(app)

# Initialize components
try:
    model = SentenceTransformer('all-MiniLM-L6-v2')
    client = QdrantClient(host='localhost', port=6333)
    collection_name = "Olabs_final"
except Exception as e:
    logging.error(f"Initialization failed: {str(e)}")
    raise

def simple_retrieval(query: str, top_k: int = 5) -> list:
    """Perform vector retrieval without metadata filtering"""
    try:
        query_embedding = model.encode(query).tolist()
        search_results = client.search(
            collection_name=collection_name,
            query_vector=query_embedding,
            limit=top_k
        )
        return [{
            "score": hit.score,
            "text": hit.payload.get("chunk_text", ""),
        } for hit in search_results]
    except Exception as e:
        logging.error(f"Search failed: {str(e)}")
        return []

@app.route('/search', methods=['POST'])
def handle_search():
    try:
        data = request.get_json()
        if not data or 'query' not in data:
            return jsonify({"error": "Missing query parameter"}), 400
            
        query = data['query']
        top_k = int(data.get('top_k', 5))
        results = simple_retrieval(query, top_k)
        
        return jsonify({
            "results": results,
            "collection": collection_name,
            "status": "success"
        })
        
    except Exception as e:
        logging.error(f"API error: {str(e)}")
        return jsonify({"error": str(e), "status": "error"}), 500

@app.route('/health', methods=['GET'])
def health_check():
    try:
        # Check Qdrant connection
        client.get_collections()
        return jsonify({
            "status": "healthy",
            "qdrant": "connected",
            "model": "loaded"
        })
    except Exception as e:
        return jsonify({
            "status": "degraded",
            "qdrant": "disconnected",
            "error": str(e)
        }), 500

if __name__ == '__main__':
    app.run(host='0.0.0.0', port=5001, debug=False, use_reloader=False)

INFO:sentence_transformers.SentenceTransformer:Use pytorch device_name: cuda
INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: all-MiniLM-L6-v2


 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5001
 * Running on http://192.168.191.180:5001
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [01/Mar/2025 14:33:42] "OPTIONS /search HTTP/1.1" 200 -


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

C:\Users\MATHAN\AppData\Local\Temp\ipykernel_20744\1716164557.py:27: DeprecationWarning: `search` method is deprecated and will be removed in the future. Use `query_points` instead.
  search_results = client.search(
ERROR:root:Search failed: [WinError 10061] No connection could be made because the target machine actively refused it
INFO:werkzeug:127.0.0.1 - - [01/Mar/2025 14:33:47] "POST /search HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [01/Mar/2025 14:38:12] "OPTIONS /search HTTP/1.1" 200 -


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

ERROR:root:Search failed: [WinError 10061] No connection could be made because the target machine actively refused it
INFO:werkzeug:127.0.0.1 - - [01/Mar/2025 14:38:16] "POST /search HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [01/Mar/2025 14:38:36] "OPTIONS /search HTTP/1.1" 200 -


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

ERROR:root:Search failed: [WinError 10061] No connection could be made because the target machine actively refused it
INFO:werkzeug:127.0.0.1 - - [01/Mar/2025 14:38:40] "POST /search HTTP/1.1" 200 -
